In [1]:
import os
import cv2
import numpy as np

# Step 1:Preparation of dataset for training
Dataset that we have downloaded have images in vaious folders,we need to have these images in two folders,one for positive images and another for negative images.
##### Image database subfolders
![Vehicle subfolder](https://github.com/khurshidrpvv/OpenCV-car-detection/blob/master/HarTraining/vehicle.PNG?raw=true)

##### Positive Images(vehicle) Subfolders
![alt text](https://github.com/khurshidrpvv/OpenCV-car-detection/blob/master/HarTraining/Positive.PNG?raw=true)

##### Negative Images(non-vehicle) Subfolders
![alt text](https://github.com/khurshidrpvv/OpenCV-car-detection/blob/master/HarTraining/Negative.PNG?raw=true)

In each of the folder, image names stars with prefix **image** and sufix four digit number starting with **0000** eg.image0000,images0001.... .In order to merger all positives in one folder and all negatives in another we need to change the names of images to avoid the merging of files with same name, and we accomplished that by changing the sufix by changing the value of i 

In [ ]:
# Renaming images according to our requirement
i = 2925
path = r'F:\Khurshid\Project\OpenCvTutorial\OwnCollection\non-vehicles\Right'
for image_name in os.listdir(path):
    old_file = os.path.join(path, image_name)
    
    if i < 1000:
        new_file = os.path.join(path,'image0'+str(i)+'.png' )
        os.rename(old_file,new_file)
    else:
        new_file = os.path.join(path,'image'+str(i)+'.png')
        os.rename(old_file,new_file)
        
    i+=1

# Step 2 : Creation of negative description file:
negative description file or commonly referred as the backgroud discription file. It is a text file that contains image name with full path.Conventionally this file is names as "bg.txt" where bg stands for backgroud.

In [11]:
def create_negative_description_file(path,file_name):
    for image in os.listdir(path):
        line = path+'\\'+image
        with open(file_name,'a') as f:
            f.write(line)
            f.write('\n')
            
create_negative_description_file(r'F:\Khurshid\Project\OpenCvTutorial\NegativeImages','bg.txt')

# Step 3: Creation of positive description file:
positive description file contains the following information.
1. full path
2. image name
3. number of object of interest in image
4. x coordinate 
5. y coordinate
6. width 
7. Hight

Our image that we downloaded has allready cropped and scaled, there is only one car in each image, (x,y) = 0, and (widht,height) = 64. So we will suffix '1 0 0 64 64 ' to create our positive description file 

In [12]:
def create_positive_description_file(path,file_name):
    #image_names = os.listdir(path)
    for image in os.listdir(path):
        line = path+'\\'+image+' 1 0 0 64 64'
        with open(file_name,'a') as f:
            f.write(line)
            f.write('\n')
            

create_positive_description_file('F:\Khurshid\Project\OpenCvTutorial\Test\PositiveImages','test.txt')            

# Step 4 : Creation of vec file:
vec file contains the information of positive images and it is needed for har training, it is a binary file.
### Steps to create vec file:
1. Go to the directory where Opencv is extracted and nevigate to 
    opencv > build > x64 > vc14 > bin 
    and copy all the files to the working directory.
    
    #### Files to copy
    ![files to copy](https://github.com/khurshidrpvv/OpenCV-car-detection/blob/master/HarTraining/vec.PNG?raw=true)
    
    In case if you couldn't find opencv folder, [download](http://opencv.org/) and extract and then follow steps
    
    #### Snapshot of working directory:
    ![working directory](https://github.com/khurshidrpvv/OpenCV-car-detection/blob/master/HarTraining/work.PNG?raw=true)
    
2. Openv cmd in working directory and type following command

    **```opencv_createsamples -info [name of positive description file] -num [number of positive images] -w [width] -h [height] -vec [name of output vec file]```**
    
    #### In our case:
    **```opencv_createsamples -info info.txt -num 1800 -w 64 -h 64 -vec cars.vec```**

This will create a vec file in working directory

# Step 5: Har training:
Open cmd in working directory and type following command:

**```opencv_traincascade -data [outputDirectory] -vec [vec file name] -bg [negative desciption file name] -numPos [number of positives] -numNeg [number of negatives] -numStages 20 -minHitRate 0.995 -maxFalseAlarmRate 0.5 -w [width] -h [height]```**

numPos is not the number or positive images used for training,instead it is calculated as:

"vec-file has to contain >= (numPose + (numStages-1) * (1 - minHitRate) * numPose) + S"

I couldn't figure out how to evaluate "S" but as generally 

numPos = 0.9*Number of positiveImages used for training

#### In our case:
numPos = 07*1800 = 762 ~ 760

numNeg = 0.7*3900 = 2730

create a XML folder in working directory,in this folder all stages xml will be saved

**```opencv_traincascade -data F:\Khurshid\Project\OpenCvTutorial\Test\XML -vec cars.vec -bg bg.txt -numPos 760 -numNeg 2730 -numStages 20 -minHitRate 0.995 -maxFalseAlarmRate 0.5 -w 64 -h 64```**

It will take minimum of 24 hours to train.Once the training is finished a xml file will be generated  